In [6]:
from p4utils.utils.helper import load_topo
from p4utils.utils.sswitch_thrift_API import SimpleSwitchThriftAPI
import argparse
import shutil
import os
import json
from pprint import pprint
from collections import defaultdict, Counter

import struct
import crcmod

In [7]:
topo = load_topo("topo/fat-tree-k4.json")

m_IS = 7
m_NS = 10

In [8]:
topo.get_switches().keys()

dict_keys(['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20'])

In [9]:
edge_switches = [sw for sw in topo.get_switches() if topo.get_hosts_connected_to(sw)]

In [10]:
topo.__dict__.keys()

dict_keys(['graph_attr_dict_factory', 'node_dict_factory', 'node_attr_dict_factory', 'adjlist_outer_dict_factory', 'adjlist_inner_dict_factory', 'edge_attr_dict_factory', 'graph', '_node', '_adj', 'edge_to_intf', 'node_to_intf', 'ip_to_host', 'nodes'])

In [13]:
all_paths = []
flow_set = set()
for src in edge_switches:
    for dst in edge_switches:
        # if src == dst: continue
        if (src, dst) in flow_set or (dst, src) in flow_set: continue
        flow_set.add((src, dst))
        # print(src, dst)
        all_paths += topo.get_shortest_paths_between_nodes(src, dst)
# all_paths = set([p[1:-1] for p in all_paths])

In [12]:
len(all_paths)

112

In [14]:
path_lens

Counter({1: 8, 3: 8, 5: 96})

In [15]:
path_lens = Counter([len(p) for p in all_paths])
path_lens

Counter({1: 8, 3: 8, 5: 96})

In [16]:
entry_num = 0
for hop_num, path_num in path_lens.items():
    entry_num += hop_num * path_num
entry_num

512

In [17]:
with open("topo/paths-k4.json", "r") as f:
    path_ctrl = json.load(f)

In [18]:
control_count = 0
for ctrl in path_ctrl.values():
    for c in ctrl['controls']:
        if c != 0:
            control_count += 1
control_count

48

In [19]:
entry_num * m_IS, control_count * m_NS

(3584, 480)

In [24]:
(entry_num * m_IS - control_count * m_NS) / (entry_num * m_IS)


0.8660714285714286